In [26]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [28]:
from crewai import Crew, Agent, Task

international_chef = Agent(
    role="International Chef",
    goal="Create ethnic cuisine recipies that are easy to cook at home",
    backstory="""
    You are an famous chef that specializes in couisine from countries all around the world.
    You know how to cook the most traditional dishes from all cultures but you also know how to adapt them for people to be able to cook them at home.
    """,
    verbose=True,
    allow_delegation=False,
)

healthy_chef = Agent(
    role="Healthy Chef",
    goal="Turn any recipe into a healthy vegetarian recipe that is easy to cook with home ingredients.",
    backstory="""
    You are a chef specialized in healthy cooking.
    You can take any recipe and change the ingredients to make it vegetarian friendly without loosing the escense of the dish and what makes it delicious.
    """,
    verbose=True,
    allow_delegation=False,
)
normal_recipe = Task(
    description="Come up with a {dish} that serves {people} people.",
    agent=international_chef,
    expected_output="Your answer MUST have three sections, the ingredients required with their quantities, the preparation instructions and serving suggestions",
    output_file="normal_recipe.md",
)

healthy_recipe = Task(
    description="Replace the ingredients of a recipe to make it vegetarian without making it less delicious, adjust if needed.",
    agent=healthy_chef,
    expected_output="Your answer MUST have four sections, the ingredients required with their quantities, the preparation instructions, serving suggestions and an explanation of the replaced ingredients.",
    output_file="healthy_recipe.md",
)
crew = Crew(
    tasks=[
        normal_recipe,
        healthy_recipe,
    ],
    agents=[
        international_chef,
        healthy_chef,
    ],
    verbose=2,
)

result = crew.kickoff(
    inputs=dict(
        dish="Greek dinner",
        people="5",
    )
)

2024-08-27 11:57:49,211 - 25892 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: International Chef
 [INFO]: == Starting Task: Come up with a Greek dinner that serves 5 people.


> Entering new CrewAgentExecutor chain...
I now can give a great answer  
Final Answer:  

### Greek Dinner for 5 People  

#### Ingredients:  
**For the Main Dish (Moussaka):**  
- 2 large eggplants, sliced into 1/4 inch rounds  
- 1 lb (450g) ground beef or lamb  
- 1 large onion, chopped  
- 2 cloves garlic, minced  
- 1 can (14 oz) crushed tomatoes  
- 2 tbsp tomato paste  
- 1 tsp ground cinnamon  
- 1 tsp dried oregano  
- 1/2 tsp allspice  
- 1/2 cup red wine (optional)  
- 4 medium potatoes, peeled and sliced into thin rounds  
- 3 cups béchamel sauce (store-bought or homemade)  
- 1 cup grated Parmesan cheese  
- Olive oil for frying  
- Salt and pepper to taste  

**For the Greek Salad:**  
- 1 large cucumber, diced  
- 2 large tomatoes, diced  
- 1 red onion, thinly sliced  
- 1 bell pepper (any color), diced  
- 1 cup Kalamata olives  
- 200g feta ch